# Character Level CNN Variational Autoencoder

In [1]:
import Corpus.gutenberg as corpus
from TextPreprocess.Tokenizer.Stanford import tokenize
from Utils.visual import hist, tally
from Utils.debug import dump
from Utils.generator import sliding_window, random_window, transform
from Utils.FS.file import save, load
from Utils.keras import compact_embedding
from Utils.misc import batch
from Utils.indexer import build_index, index_2_one_hot
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
data = tokenize(corpus.raw())

In [3]:
def char_generator():
    for word in data:
        for char in word:
            yield char

In [4]:
s2i, i2s, size = build_index(char_generator())

In [5]:
MAX_SEQ_LENGTH = max([len(word) for word in data])
MAX_SEQ_LENGTH

48

In [6]:
NUM_SYMBOL = size
NUM_SYMBOL

87

In [7]:
def word_generator():
    for word in data:
            yield word

NUM_SAMPLE = len(list(word_generator()))
NUM_SAMPLE

2544236

In [85]:
from keras.layers import Input, Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, Reshape, Embedding
from keras.models import Model, Sequential
from keras.regularizers import l2
import numpy as np
import tensorflow as tf

def custom_loss(y_true, y_pred):
    print(y_true, y_pred)
    '''Just another crossentropy'''
    #y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    #y_pred /= y_pred.sum(axis=-1, keepdims=True)
    #cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    '''
    [np.average
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=y_true[i],
            logits=y_pred[i],
        )
     for i in y_true]
    '''
    return y_true - y_pred

def create_baseline(dropout=0, branching=5):
    
    inp = Input(shape=(MAX_SEQ_LENGTH,NUM_SYMBOL))
    #x = Embedding(NUM_SYMBOL, NUM_SYMBOL, weights=[np.eye(NUM_SYMBOL)], input_length=10, trainable=False)(inp)
    x = Reshape((1, MAX_SEQ_LENGTH, NUM_SYMBOL))(inp)
    x = Conv2D(300, (1, 2), use_bias=False)(x)
    x = Conv2D(10, (1, 2), activation='selu', use_bias=False)(x)
    x = Conv2DTranspose(300, (1, 2), use_bias=False)(x)
    x = Conv2DTranspose(NUM_SYMBOL, (1, 2), use_bias=False)(x)
    x = Reshape((MAX_SEQ_LENGTH, NUM_SYMBOL))(x)
    model = Model(inp, x)
    model.compile(loss='mean_squared_error',
              optimizer='adam')
    return model

In [86]:
from keras_tqdm import TQDMNotebookCallback

BATCH_SIZE = 1024
DROPOUT = 0.5
BRANCHING = 2

model = create_baseline(DROPOUT, BRANCHING)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 48, 87)            0         
_________________________________________________________________
reshape_33 (Reshape)         (None, 1, 48, 87)         0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 1, 47, 300)        52200     
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 1, 46, 10)         6000      
_________________________________________________________________
conv2d_transpose_33 (Conv2DT (None, 1, 47, 300)        6000      
_________________________________________________________________
conv2d_transpose_34 (Conv2DT (None, 1, 48, 87)         52200     
_________________________________________________________________
reshape_34 (Reshape)         (None, 48, 87)            0         
Total para

In [87]:
from keras.utils.np_utils import to_categorical
def sample_generator(word_generator, batch_size = 64):
    sample = []
    for word in word_generator:
        unpad = batch(s2i, word)
        padded = pad_sequences([unpad], maxlen=MAX_SEQ_LENGTH, dtype='float32', padding='pre', truncating='pre', value=0.)
        one_hot = to_categorical(padded, num_classes=NUM_SYMBOL)
        sample.append(one_hot)
        if len(sample) == batch_size:
            train = np.array(sample)
            yield(train, train)
            sample = []

In [88]:
word_gen = word_generator()
next(sample_generator(word_gen, 2))[0].shape

(2, 48, 87)

In [89]:
word_gen = word_generator()
BATCH_SIZE = 128
model.fit_generator(
    sample_generator(word_gen, BATCH_SIZE),
    NUM_SAMPLE // BATCH_SIZE,
    epochs=200,
    #class_weight=class_weight
    #verbose=0, callbacks=[TQDMNotebookCallback()]
)

Epoch 1/200
  293/19876 [..............................] - ETA: 3239s - loss: 8.4559e-04

KeyboardInterrupt: 